In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

c:\Users\Ashwin\miniconda3\envs\tf_gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import os
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [3]:
test=pd.read_csv("../input/test_user_item.csv")
train=pd.read_csv("../input/train_user_item.csv")
validation=pd.read_csv("../input/validation_user_item.csv")

In [25]:
# bert=tf.keras.models.load_model("../models/bert_en_uncased_model/")
bert=hub.load("../models/bert_en_uncased_model/")

In [27]:
bert

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x1f657b5aa60>

In [30]:
bert_preprocessing=hub.load("../models/bert_en_uncased_preprocess")

In [31]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocessing(text_test)

In [32]:
text_preprocessed

{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[ 101, 2023, 2003, 2107, 2019, 6429, 3185,  999,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,   

In [33]:
bert=hub.KerasLayer(bert)

In [34]:
bert.trainable

False

In [35]:
bert(text_preprocessed)

{'pooled_output': <tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-0.9216864 , -0.39364985, -0.5401813 ,  0.68267155,  0.4390489 ,
         -0.1402008 ,  0.87749565,  0.2605826 , -0.6316952 , -0.9999659 ,
         -0.263578  ,  0.85122913,  0.98570925,  0.18641602,  0.96457815,
         -0.60037684, -0.2161695 , -0.60723436,  0.33560044, -0.5507246 ,
          0.7183043 ,  0.99972135,  0.399579  ,  0.2708195 ,  0.48029974,
          0.9555295 , -0.77164483,  0.9614904 ,  0.95828795,  0.72736114,
         -0.7411457 ,  0.11925898, -0.9911773 , -0.22011821, -0.68260324,
         -0.99011135,  0.36183155, -0.7947883 ,  0.07121448,  0.0930822 ,
         -0.9396649 ,  0.1408429 ,  0.99987   , -0.349242  ,  0.13441902,
         -0.33571568, -0.99999964,  0.18737721, -0.87942606,  0.6794094 ,
          0.5336223 ,  0.41188294,  0.16478644,  0.46623108,  0.3736972 ,
         -0.05967636, -0.17970116,  0.02793555, -0.18826607, -0.5940139 ,
         -0.61240923,  0.27810332, -0.751990

In [ ]:
def create_model(embedding_size=16):
    user_tag=Input(name="user_tags",shape=[1],dtype=tf.string)
    movie_tag=Input(name="movie_tags",shape=[1],dtype=tf.string)

    user_tag_vectorized = TextVectorization(
    standardize="strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH,
    vocabulary=VOCABULARY)(user_tag)
    
    movie_tag_vectorized = TextVectorization(
    standardize="strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH,
    vocabulary=VOCABULARY)(movie_tag)
    
    user_embedding= Embedding(name = 'user_embedding',input_dim = NUMBER_OF_USERS, output_dim = embedding_size)(user_tag_vectorized)
    movie_embedding= Embedding(name = 'movie_embedding',input_dim = NUMBER_OF_MOVIES, output_dim = embedding_size)(movie_tag_vectorized)
    
    merged = Dot(name = 'dot_product', normalize = True, axes = 2)([user_embedding, movie_embedding])
    
    merged = Reshape(target_shape = [2500])(merged)
    
    merged=Dense(2500, activation="elu")(merged)
    
    merged=Dense(1500, activation="elu")(merged)
    
    merged=Dense(500, activation="elu")(merged)
    
    out = Dense(6, activation="softmax")(merged)
    
    model = Model(inputs = [user_tag, movie_tag], outputs = out)
    
    optimizer = tf.keras.optimizers.Adam(clipvalue=1.0)
    
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model